In [ ]:
from pycocotools.coco import COCO
import requests
import os
from os.path import join
from tqdm import tqdm
import json
import pandas as pd 

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2021-11-15 15:35:42--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.139.49
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.139.49|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  46.0MB/s    in 5.8s    

2021-11-15 15:35:48 (41.8 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
!unzip annotations_trainval2017.zip

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [ ]:
!git clone https://github.com/ssaru/convert2Yolo

Cloning into 'convert2Yolo'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 215 (delta 4), reused 6 (delta 4), pack-reused 201
Receiving objects: 100% (215/215), 1002.72 KiB | 21.80 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [ ]:
!mkdir convert2Yolo/COCO 
!mkdir convert2Yolo/COCO/val2017
!mkdir convert2Yolo/COCO/annotations
!mkdir convert2Yolo/YOLO/
!mkdir convert2Yolo/YOLO/car/

In [ ]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2021-11-15 15:56:37--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2021-11-15 15:56:37 (21.9 MB/s) - ‘coco.names’ saved [625/625]



In [ ]:
!mv /content/coco.names convert2Yolo/COCO 

In [ ]:
!mv /content/selected_images/* convert2Yolo/COCO/val2017

mv: cannot stat '/content/selected_images/*': No such file or directory


In [ ]:
!mv /content/annotations/instances_train2017.json /content/convert2Yolo/COCO/annotations

In [ ]:
!rm -rf /content/convert2Yolo/COCO/annotations/truck_train

In [ ]:
!python convert2Yolo/example.py --datasets COCO --img_path convert2Yolo/COCO/val2017/ --label convert2Yolo/COCO/annotations/class_annotation/car.json --convert_output_path convert2Yolo/YOLO/car/ --img_type ".jpg" --manifest_path convert2Yolo/ --cls_list_file convert2Yolo/COCO/coco.names



COCO Parsing:  |████████████████████████████████████████| 100.0% (43867/43867)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (12251/12251)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (12251/12251)  Complete



In [ ]:
!mkdir /content/convert2Yolo/YOLO/bus
!mkdir /content/convert2Yolo/YOLO/truck

In [ ]:
!python convert2Yolo/example.py --datasets COCO --img_path convert2Yolo/COCO/val2017/ --label convert2Yolo/COCO/annotations/class_annotation/bus.json --convert_output_path convert2Yolo/YOLO/bus/ --img_type ".jpg" --manifest_path convert2Yolo/ --cls_list_file convert2Yolo/COCO/coco.names



COCO Parsing:  |████████████████████████████████████████| 100.0% (6069/6069)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (3952/3952)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (3952/3952)  Complete



In [ ]:
!python convert2Yolo/example.py --datasets COCO --img_path convert2Yolo/COCO/val2017/ --label convert2Yolo/COCO/annotations/class_annotation/truck.json --convert_output_path convert2Yolo/YOLO/truck/ --img_type ".jpg" --manifest_path convert2Yolo/ --cls_list_file convert2Yolo/COCO/coco.names



COCO Parsing:  |████████████████████████████████████████| 100.0% (9973/9973)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (6127/6127)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (6127/6127)  Complete



## 2. Pobranie 2000 zdjec z kazdej klasy pojazdów wraz z jej anotacjami w formacie yolo

### Cars

In [ ]:
#create directories for our custom dataset 
!mkdir /content/vehicles
!mkdir /content/vehicles/car
!mkdir /content/vehicles/truck
!mkdir /content/vehicles/bus

In [ ]:
def prepare_sliced_images(category):
  coco = COCO('/content/convert2Yolo/COCO/annotations/instances_train2017.json')
  # wyspecyfikowanie konkretnych kategorii
  catIds = coco.getCatIds(catNms=[category])

  # wybierz pliki z id odpowiadającej wczesniej wybranej kategori
  imgIds = coco.getImgIds(catIds=catIds)
  images = coco.loadImgs(imgIds)

  # amonge all images choose 2000 
  return images[0:2000]

In [ ]:
sliced_cars = prepare_sliced_images('car')
sliced_buses = prepare_sliced_images('bus')
sliced_trucks = prepare_sliced_images('truck')

loading annotations into memory...
Done (t=27.01s)
creating index...
index created!
loading annotations into memory...
Done (t=23.25s)
creating index...
index created!
loading annotations into memory...
Done (t=20.61s)
creating index...
index created!


In [ ]:
def save_image_for_category(category, sliced_images):
  for im in sliced_images:
      img_data = requests.get(im['coco_url']).content
      with open(f"/content/vehicles/{category}/{im['file_name']}", 'wb') as handler:
          handler.write(img_data)

In [ ]:
nazwa = sliced_trucks[0]['file_name']

In [ ]:
print(nazwa.replace(".jpg", ".txt"))

000000262148.txt


In [ ]:
save_image_for_category('car', sliced_cars)
save_image_for_category('bus', sliced_buses)
save_image_for_category('truck', sliced_trucks)

In [ ]:
#create directories for our yolo annotations
!mkdir /content/vehicles_labels/
!mkdir /content/vehicles_labels/car
!mkdir /content/vehicles_labels/truck
!mkdir /content/vehicles_labels/bus

In [ ]:
def create_images_list(category, sliced_images):
  f = open(f'labels_{category}_list.txt', "w")

  for im in range(len(sliced_images)):
      image = sliced_images[im]['file_name'].replace(".jpg", ".txt\n")
      f.writelines(image)

  f.close()

In [ ]:
create_images_list('car', sliced_cars)

In [ ]:
create_images_list('bus', sliced_buses)

In [ ]:
create_images_list('truck', sliced_trucks)

In [ ]:
import os
import shutil

source = "/content/convert2Yolo/YOLO/car/"
destination = "/content/vehicles_labels/car/"

file1 = open('labels_car_list.txt', 'r')
files = file1.readlines()

for file_name in files:
    name = file_name
    sliced_name = name[:-1]
    file_source = source + sliced_name
    dest = shutil.copy(file_source, destination)

In [ ]:
import os
import shutil

source = "/content/convert2Yolo/YOLO/bus/"
destination = "/content/vehicles_labels/bus/"

file1 = open('labels_bus_list.txt', 'r')
files = file1.readlines()

for file_name in files:
    name = file_name
    sliced_name = name[:-1]
    file_source = source + sliced_name
    dest = shutil.copy(file_source, destination)

In [ ]:
import os
import shutil

source = "/content/convert2Yolo/YOLO/truck/"
destination = "/content/vehicles_labels/truck/"

file1 = open('labels_truck_list.txt', 'r')
files = file1.readlines()

for file_name in files:
    name = file_name
    sliced_name = name[:-1]
    file_source = source + sliced_name
    dest = shutil.copy(file_source, destination)

##3. Zapisanie etykiet oraz zdjec i pobranie

In [ ]:
!zip -r vehicle_images.zip /content/vehicles/*

Streaming output truncated to the last 5000 lines.
  adding: content/vehicles/bus/000000388026.jpg (deflated 5%)
  adding: content/vehicles/bus/000000363111.jpg (deflated 0%)
  adding: content/vehicles/bus/000000042800.jpg (deflated 0%)
  adding: content/vehicles/bus/000000379052.jpg (deflated 0%)
  adding: content/vehicles/bus/000000061182.jpg (deflated 1%)
  adding: content/vehicles/bus/000000067178.jpg (deflated 0%)
  adding: content/vehicles/bus/000000026274.jpg (deflated 0%)
  adding: content/vehicles/bus/000000436406.jpg (deflated 0%)
  adding: content/vehicles/bus/000000353276.jpg (deflated 0%)
  adding: content/vehicles/bus/000000166529.jpg (deflated 0%)
  adding: content/vehicles/bus/000000387515.jpg (deflated 0%)
  adding: content/vehicles/bus/000000542307.jpg (deflated 4%)
  adding: content/vehicles/bus/000000393403.jpg (deflated 0%)
  adding: content/vehicles/bus/000000280733.jpg (deflated 0%)
  adding: content/vehicles/bus/000000347784.jpg (deflated 0%)
  adding: content/v

In [ ]:
!zip -r vehicle_labels.zip /content/vehicles_labels/*

Streaming output truncated to the last 5000 lines.
updating: content/vehicles_labels/bus/000000116848.txt (deflated 12%)
updating: content/vehicles_labels/bus/000000437262.txt (deflated 8%)
updating: content/vehicles_labels/bus/000000262442.txt (deflated 23%)
updating: content/vehicles_labels/bus/000000084499.txt (deflated 8%)
updating: content/vehicles_labels/bus/000000196811.txt (deflated 8%)
updating: content/vehicles_labels/bus/000000271441.txt (deflated 15%)
updating: content/vehicles_labels/bus/000000567665.txt (deflated 20%)
updating: content/vehicles_labels/bus/000000527090.txt (deflated 12%)
updating: content/vehicles_labels/bus/000000273067.txt (deflated 12%)
updating: content/vehicles_labels/bus/000000224790.txt (deflated 20%)
updating: content/vehicles_labels/bus/000000353809.txt (deflated 12%)
updating: content/vehicles_labels/bus/000000453002.txt (deflated 21%)
updating: content/vehicles_labels/bus/000000353473.txt (deflated 48%)
updating: content/vehicles_labels/bus/0000

In [ ]:
!zip -r vehicle_images_car.zip /content/vehicles/car/*

  adding: content/vehicles/car/000000000064.jpg (deflated 0%)
  adding: content/vehicles/car/000000000071.jpg (deflated 0%)
  adding: content/vehicles/car/000000000094.jpg (deflated 0%)
  adding: content/vehicles/car/000000000149.jpg (deflated 0%)
  adding: content/vehicles/car/000000000247.jpg (deflated 0%)
  adding: content/vehicles/car/000000000260.jpg (deflated 0%)
  adding: content/vehicles/car/000000000307.jpg (deflated 0%)
  adding: content/vehicles/car/000000000328.jpg (deflated 1%)
  adding: content/vehicles/car/000000000359.jpg (deflated 2%)
  adding: content/vehicles/car/000000000419.jpg (deflated 0%)
  adding: content/vehicles/car/000000000540.jpg (deflated 0%)
  adding: content/vehicles/car/000000000650.jpg (deflated 1%)
  adding: content/vehicles/car/000000000656.jpg (deflated 0%)
  adding: content/vehicles/car/000000000722.jpg (deflated 0%)
  adding: content/vehicles/car/000000000730.jpg (deflated 0%)
  adding: content/vehicles/car/000000000731.jpg (deflated 0%)
  adding

In [ ]:
!zip -r vehicle_images_truck.zip /content/vehicles/truck/*

  adding: content/vehicles/truck/000000000064.jpg (deflated 0%)
  adding: content/vehicles/truck/000000000071.jpg (deflated 0%)
  adding: content/vehicles/truck/000000000094.jpg (deflated 0%)
  adding: content/vehicles/truck/000000000257.jpg (deflated 0%)
  adding: content/vehicles/truck/000000000540.jpg (deflated 0%)
  adding: content/vehicles/truck/000000000839.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001102.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001171.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001315.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001381.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001451.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001518.jpg (deflated 1%)
  adding: content/vehicles/truck/000000001523.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001554.jpg (deflated 0%)
  adding: content/vehicles/truck/000000001590.jpg (deflated 0%)
  adding: content/vehicles/truck/0000000

In [ ]:
!zip -r vehicle_images_bus.zip /content/vehicles/bus/*

  adding: content/vehicles/bus/000000000260.jpg (deflated 0%)
  adding: content/vehicles/bus/000000000471.jpg (deflated 1%)
  adding: content/vehicles/bus/000000000532.jpg (deflated 0%)
  adding: content/vehicles/bus/000000000540.jpg (deflated 0%)
  adding: content/vehicles/bus/000000000641.jpg (deflated 0%)
  adding: content/vehicles/bus/000000000681.jpg (deflated 1%)
  adding: content/vehicles/bus/000000000730.jpg (deflated 0%)
  adding: content/vehicles/bus/000000000985.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001307.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001315.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001366.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001523.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001625.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001720.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001762.jpg (deflated 0%)
  adding: content/vehicles/bus/000000001875.jpg (deflated 2%)
  adding